<a href="https://colab.research.google.com/github/anasmghouri/Coursera_Capstone/blob/main/The_Battle_of_Neighborhoods_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>